In [1]:
from pathlib import Path
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.preprocessing import Normalizer
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate, GridSearchCV, StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import LabelEncoder
import sys
import csv
import csv
import os

## Import  and merge Files

In [3]:
path = "data_right_price/data_localisee/"

def sort_csv_files(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    dfs = []
    for file in files:
        df = pd.read_csv(os.path.join(folder_path, file))
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

df = sort_csv_files(path)

/var/folders/s5/c05fkgj90cz2t3tmp495bdfc0000gn/T/ipykernel_63830/168391431.py:11: DtypeWarning: Columns (60) have mixed types.Specify dtype option on import or set low_memory=False.
  df = sort_csv_files(path)


In [42]:
df_bis = sort_csv_files(path)

/var/folders/s5/c05fkgj90cz2t3tmp495bdfc0000gn/T/ipykernel_63830/1767232073.py:1: DtypeWarning: Columns (60) have mixed types.Specify dtype option on import or set low_memory=False.
  df_bis = sort_csv_files(path)


[LightGBM] [Warning] Unknown parameter: bootstrap_features
[LightGBM] [Warning] Unknown parameter: bootstrap
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: bootstrap_features
[LightGBM] [Warning] Unknown parameter: bootstrap
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: bootstrap_features
[LightGBM] [Warning] Unknown parameter: bootstrap
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: bootstrap_features
[LightGBM] [Warning] Unknown parameter: bootstrap
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: bootstrap_features
[LightGBM] [Warning] Unknown parameter: bootstrap
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: bootstrap_features
[LightGBM] [Warning] Unknown parameter: bootstrap
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown

In [4]:
#Cesar

#def sort_csv_files(folder_path):
 # files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
  #dfs = []
#
 # for file in files:
  #    df = pd.read_csv(os.path.join(folder_path, file))
   #   dfs.append(df)
#
 # return pd.concat(dfs, ignore_index=True)

In [5]:
#Cesar
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

## Data exploration

In [6]:
df.columns

Index(['Unnamed: 0', 'idmutation', 'idmutinvar', 'idopendata', 'idnatmut',
       'datemut', 'libnatmut', 'valeurfonc', 'nbdispo', 'nblot', 'nbcomm',
       'l_codinsee', 'nbsection', 'l_section', 'nbpar', 'l_idpar', 'nbparmut',
       'l_idparmut', 'nbsuf', 'sterr', 'nbvolmut', 'nblocmut', 'l_idlocmut',
       'nblocmai', 'nblocapt', 'nblocdep', 'nblocact', 'nbapt1pp', 'nbapt2pp',
       'nbapt3pp', 'nbapt4pp', 'nbapt5pp', 'nbmai1pp', 'nbmai2pp', 'nbmai3pp',
       'nbmai4pp', 'nbmai5pp', 'sbati', 'sbatmai', 'sbatapt', 'sbatact',
       'sapt1pp', 'sapt2pp', 'sapt3pp', 'sapt4pp', 'sapt5pp', 'smai1pp',
       'smai2pp', 'smai3pp', 'smai4pp', 'smai5pp', 'codtypbien', 'libtypbien',
       'latitude', 'longitude', 'Dept', 'day', 'weekday', 'month', 'year',
       'day_month', 'Unnamed: 0.1', 'codservch', 'refdoc', 'anneemut',
       'moismut', 'coddep', 'vefa', 'index', 'geometry'],
      dtype='object')

In [7]:
df = df.drop(['Unnamed: 0.1','Unnamed: 0','codservch','refdoc'], axis=1)

### Missing values

In [8]:
#Cesar
features_nan=[feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtypes=='O']

for feature in features_nan:
    print("{}: {}% missing values".format(feature,np.round(df[feature].isnull().mean(),4)))

datemut: 0.0004% missing values
day_month: 0.9969% missing values
vefa: 0.4998% missing values
geometry: 0.9996% missing values


In [9]:
#Cesar
## Now lets check for numerical variables the contains missing values
numerical_with_nan=[feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtypes!='O']

## We will print the numerical nan variables and percentage of missing values

for feature in numerical_with_nan:
    print("{}: {}% missing value".format(feature,np.around(df[feature].isnull().mean(),4)))

valeurfonc: 0.0008% missing value
Dept: 0.4998% missing value
day: 0.4998% missing value
weekday: 0.4998% missing value
month: 0.4998% missing value
year: 0.4998% missing value
anneemut: 0.5002% missing value
moismut: 0.5002% missing value
coddep: 0.5002% missing value
index: 0.9996% missing value


In [10]:
#Cesar
## Replacing the numerical Missing Values

for feature in numerical_with_nan:
    ## We will replace by using median since there are outliers
    median_value=df[feature].median()
    
    ## create a new feature to capture nan values
    df[feature+'nan']=np.where(df[feature].isnull(),1,0)
    df[feature].fillna(median_value,inplace=True)
    
df[numerical_with_nan].isnull().sum()

valeurfonc    0
Dept          0
day           0
weekday       0
month         0
year          0
anneemut      0
moismut       0
coddep        0
index         0
dtype: int64

In [11]:
df_num = df.select_dtypes(exclude=["object"])
df_num.head()

,idmutation,idnatmut,valeurfonc,nbdispo,nblot,nbcomm,nbsection,nbpar,nbparmut,nbsuf,...,valeurfoncnan,Deptnan,daynan,weekdaynan,monthnan,yearnan,anneemutnan,moismutnan,coddepnan,indexnan
0,6539278.0,1.0,650340.0,1.0,4.0,1.0,1.0,1.0,0.0,0.0,...,0,0,0,0,0,0,1,1,1,1
1,6660956.0,1.0,380000.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,...,0,0,0,0,0,0,1,1,1,1
2,6374603.0,1.0,105000.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0,0,0,0,0,0,1,1,1,1
3,6393753.0,1.0,415000.0,1.0,4.0,1.0,1.0,1.0,0.0,0.0,...,0,0,0,0,0,0,1,1,1,1
4,7090098.0,1.0,90000.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0,0,0,0,0,0,1,1,1,1


In [ ]:
def preprocessing(df):
    df = df.drop(['Unnamed: 0.1','Unnamed: 0','codservch','refdoc'], axis=1)
    
    features_nan=[feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtypes=='O']
    numerical_with_nan=[feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtypes!='O']
    
    for feature in numerical_with_nan:
        ## We will replace by using median since there are outliers
        median_value=df[feature].median()

        ## create a new feature to capture nan values
        df[feature+'nan']=np.where(df[feature].isnull(),1,0)
        df[feature].fillna(median_value,inplace=True)
    
    df_num = df.select_dtypes(exclude=["object"])
    
    
    sel = VarianceThreshold(threshold=0.05) # 0.05: drop column where 95% of the values are constant
    # fit finds the features with constant variance
    sel.fit(df_num.iloc[:, :-1])
    quasi_constant_features_list = [x for x in df_num.iloc[:, :-1].columns if x not in df_num.iloc[:, :-1].columns[sel.get_support()]]
    df_num.drop(quasi_constant_features_list, axis=1, inplace=True)
    
    

In [12]:
#Cesar
# Let's drop quasi-constant features where 95% of the values are similar or constant
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0.05) # 0.05: drop column where 95% of the values are constant

# fit finds the features with constant variance
sel.fit(df_num.iloc[:, :-1])


# Get the number of features that are not constant
print(f"Number of retained features: {sum(sel.get_support())}")
print(
    f"\nNumber of quasi_constant features: {len(df_num.iloc[:, :-1].columns) - sum(sel.get_support())}")

quasi_constant_features_list = [x for x in df_num.iloc[:, :-
                                                             1].columns if x not in df_num.iloc[:, :-1].columns[sel.get_support()]]

print(
    f"\nQuasi-constant features to be dropped: {quasi_constant_features_list}")


# Let's drop these columns from df_train_num
df_num.drop(quasi_constant_features_list, axis=1, inplace=True)

Number of retained features: 56

Number of quasi_constant features: 6

Quasi-constant features to be dropped: ['nbcomm', 'nbsection', 'nbmai1pp', 'nbmai2pp', 'latitude', 'valeurfoncnan']


/Users/augustin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [13]:
#Cesar
# Let's select features where the correlation with 'SalePrice' is higher than |0.3|
# -1 because the latest row is SalePrice
df_num_corr = df_num.corr()["valeurfonc"][:-1]

# Correlated features (r2 > 0.5)
high_features_list = df_num_corr[abs(
    df_num_corr) >= 0.5].sort_values(ascending=False)
print(
    f"{len(high_features_list)} strongly correlated values with Sale Price (valeur fonc):\n{high_features_list}\n")

# Correlated features (0.3 < r2 < 0.5)
low_features_list = df_num_corr[(abs(df_num_corr) < 0.5) & (
    abs(df_num_corr) >= 0.3)].sort_values(ascending=False)
print(
    f"{len(low_features_list)} slightly correlated values with Sale Price (valeur fonc):\n{low_features_list}")

1 strongly correlated values with Sale Price (valeur fonc):
valeurfonc    1.0
Name: valeurfonc, dtype: float64

0 slightly correlated values with Sale Price (valeur fonc):
Series([], Name: valeurfonc, dtype: float64)


In [14]:
import seaborn as sns
# With the boxplot we can see the variation of the target 'SalePrice' in each of the categorical features
#fig, axes = plt.subplots(
 #   round(len(df_num.columns)/3), 3, figsize=(15, 30))

#for i, ax in enumerate(fig.axes):
    # plot the variation of SalePrice in each feature
 #   if i < len(df_num.columns) - 1:
  #      ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=45)
   #     sns.boxplot(
    #        x=df_num.columns[i], y="valeurfonc", data=df_num, ax=ax)

#fig.tight_layout()

## Preprocessing

In [ ]:
path = "data_right_price/data_localisee/"

def sort_csv_files(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    dfs = []
    for file in files:
        df = pd.read_csv(os.path.join(folder_path, file))
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

df = sort_csv_files(path)

In [15]:
def exclude_lines(df, max_fonc=1500000):
    res = df[df["valeurfonc"] <= max_fonc]
    res = res[res["valeurfonc"] >= 100]
    res = res[((res["libtypbien"] == 'UN APPARTEMENT')
               | (res["libtypbien"] == 'PPARTEMENT INDETERMINE')
               | (res["libtypbien"] == 'UNE MAISON')
               | (res["libtypbien"] == 'MAISON - INDETERMINEE')
               | (res["libtypbien"] == 'DES MAISONS')
               | (res["libtypbien"] == 'DEUX APPARTEMENTS')
              )]
    return res

df = exclude_lines(df)

In [16]:
def feature_selection(df):
    base_columns = ["anneemut", "moismut", "coddep", "vefa", "valeurfonc", "sterr",
                    "nblocdep", "latitude", "longitude", "nb_accomodations", "surf",
                    "arro", "nb_rooms"
                   ]
    df["nb_accomodations"] = df["nblocmai"] + df["nblocapt"]
    df["surf"] = df["sbatmai"] + df["sbatapt"] #df["sbati"] + df["sbatact"]
    df["arro"] = df["l_codinsee"].apply(lambda s: s[-4:-2] if s[2:4]=="75" else 0).astype("int")
    df["nb_rooms"] = (df["nbapt1pp"] 
                       + 2 * df["nbapt2pp"]
                       + 3 * df["nbapt3pp"]
                       + 4 * df["nbapt4pp"]
                       + 5 * df["nbapt5pp"]
                       + df["nbmai1pp"] 
                       + 2 * df["nbmai2pp"]
                       + 3 * df["nbmai3pp"]
                       + 4 * df["nbmai4pp"]
                       + 5 * df["nbmai5pp"]
                      ) / df["nb_accomodations"]
    return df[base_columns]

df = feature_selection(df)

In [17]:
obj_cols = df.select_dtypes(include='object').columns

# Initialize LabelEncoder
le = LabelEncoder()

# Apply label encoding to all object type columns
for col in obj_cols:
    df[col] = le.fit_transform(df[col])

In [65]:
from sklearn.preprocessing import MinMaxScaler
import sklearn
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import joblib

def sort_csv_files(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    dfs = []
    for file in files:
        df = pd.read_csv(os.path.join(folder_path, file))
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

def exclude_lines(df, max_fonc=1500000):
    res = df[df["valeurfonc"] <= max_fonc]
    res = res[res["valeurfonc"] >= 100]
    res = res[((res["libtypbien"] == 'UN APPARTEMENT')
               | (res["libtypbien"] == 'PPARTEMENT INDETERMINE')
               | (res["libtypbien"] == 'UNE MAISON')
               | (res["libtypbien"] == 'MAISON - INDETERMINEE')
               | (res["libtypbien"] == 'DES MAISONS')
               | (res["libtypbien"] == 'DEUX APPARTEMENTS')
              )]
    return res

def feature_selection(df):
    base_columns = ["anneemut", "moismut", "coddep", "vefa", "valeurfonc", "sterr",
                    "nblocdep", "latitude", "longitude", "nb_accomodations", "surf",
                    "arro", "nb_rooms"
                   ]
    df["nb_accomodations"] = df["nblocmai"] + df["nblocapt"]
    df["surf"] = df["sbatmai"] + df["sbatapt"] #df["sbati"] + df["sbatact"]
    df["arro"] = df["l_codinsee"].apply(lambda s: s[-4:-2] if s[2:4]=="75" else 0).astype("int")
    df["nb_rooms"] = (df["nbapt1pp"] 
                       + 2 * df["nbapt2pp"]
                       + 3 * df["nbapt3pp"]
                       + 4 * df["nbapt4pp"]
                       + 5 * df["nbapt5pp"]
                       + df["nbmai1pp"] 
                       + 2 * df["nbmai2pp"]
                       + 3 * df["nbmai3pp"]
                       + 4 * df["nbmai4pp"]
                       + 5 * df["nbmai5pp"]
                      ) / df["nb_accomodations"]
    return df[base_columns]

def filter_valfonc(df):
    mutation = df
    mutation = mutation[~mutation["valeurfonc"].isna()]
    mutation = mutation[mutation["valeurfonc"] >= 1000]
    mutation = mutation[mutation["valeurfonc"] <= 1000000]
    return mutation

def add_arrondissement(df):
    n_col = df["l_codinsee"].apply(lambda s: s[-4:-2] if s[2:4]=="75" else 0)
    #n_col = n_col.rename("arro")
    df["arro"] = n_col.astype("int")

def sum_surface(df):
    df["surf"] = (df["sbati"] 
                  + df["sbatmai"] 
                  + df["sbatapt"]
                  + df["sbatact"])
    
def min_max_scale(df):
    scaler = StandardScaler()
    df['valeurfonc'] = scaler.fit_transform(df['valeurfonc'].values.reshape(-1, 1))

def X_y_split(df):
    features = ["anneemut", "moismut", "nblot", "sterr", "nbvolmut", "nblocmut",
                "nblocmai", "nblocapt", "nblocdep", "nblocact", "latitude", "longitude",
                "arro", "surf"
               ]
    X = df[features]
    y = df["valeurfonc"]
    return X, y

def training(X_train, Y_train):  
    xgboost = XGBRegressor()
    xgboost.fit(X_train, Y_train)

def predict(X_test):
    y_pred = xgboost.predict(X_test)
    return y_pred

def save_model(pkl_file_path):
    joblib.dump(xgboost, pkl_file_path)
    
def load_model(pkl_file_path):
    xgboost = joblib.load(pkl_file_path)
    return xgboost

In [53]:
def sort_csv_files(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    dfs = []
    for file in files:
        df = pd.read_csv(os.path.join(folder_path, file))
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

In [54]:
def filter_valfonc(df):
    mutation = df
    mutation = mutation[~mutation["valeurfonc"].isna()]
    mutation = mutation[mutation["valeurfonc"] >= 1000]
    mutation = mutation[mutation["valeurfonc"] <= 1000000]
    return mutation

In [55]:
def add_arrondissement(df):
    n_col = df["l_codinsee"].apply(lambda s: s[-4:-2] if s[2:4]=="75" else 0)
    #n_col = n_col.rename("arro")
    df["arro"] = n_col.astype("int")

def sum_surface(df):
    df["surf"] = (df["sbati"] 
                        + df["sbatmai"] 
                        + df["sbatapt"]
                        + df["sbatact"])

In [56]:
def min_max_scale(df):
    scaler = StandardScaler()
    df['valeurfonc'] = scaler.fit_transform(df['valeurfonc'].values.reshape(-1, 1))

In [57]:
def X_y_split(df):
    features = ["anneemut", "moismut", "nblot", "sterr", "nbvolmut", "nblocmut",
                "nblocmai", "nblocapt", "nblocdep", "nblocact", "latitude", "longitude",
                "arro", "surf"
               ]
    X = df[features]
    y = df["valeurfonc"]
    return X, y

In [59]:
def training(X_train, Y_train):  
    xgboost = XGBRegressor()
    xgboost.fit(X_train, Y_train)

def predict(X_test):
    y_pred = xgboost.predict(X_test)
    return y_pred

In [62]:
def save_model(pkl_file_path):
    joblib.dump(xgboost, pkl_file_path)

In [67]:
def load_model(pkl_file_path):
    xgboost = joblib.load(pkl_file_path)
    return xgboost

In [68]:
# Let's define a function for each metrics
# R²
def rsqr_score(test, pred):
    """Calculate Root Mean Square Error score 

    Args:
        test -- test data
        pred -- predicted data

    Returns:
        Root Mean Square Error score
    """
    rsqr_ = r2_score(test, pred)
    return rsqr_
def rmse_score(test, pred):
    """Calculate Root Mean Square Error score 

    Args:
        test -- test data
        pred -- predicted data

    Returns:
        Root Mean Square Error score
    """
    rmse_ = np.sqrt(mean_squared_error(test, pred))
    return rmse_

def mse_score(test, pred):
    """Calculate Root Mean Square Error score 

    Args:
        test -- test data
        pred -- predicted data

    Returns:
        Root Mean Square Error score
    """
    mse_ = mean_squared_error(test, pred)
    return mse_

# Print the scores
def print_score(test, pred):
    """Print calculated score 

    Args:
        test -- test data
        pred -- predicted data

    Returns:
        print the regressor name
        print the R squared score
        print Root Mean Square Error score
        print Mean Square Error score
    """

    #print(f"- Regressor: {regr.__class__.__name__}")
    print(f"R²: {rsqr_score(test, pred)}")
    print(f"RMSE: {rmse_score(test, pred)}")
    print(f"MSE: {mse_score(test, pred)}")

In [61]:
from time import time

start = time()

path = "data_right_price/data_localisee/"
df_1 = sort_csv_files(path)
mutation = filter_valfonc(df_1)
add_arrondissement(mutation)
sum_surface(mutation)
min_max_scale(mutation)
X, y = X_y_split(mutation)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 100)
training(X_train, Y_train)
y_pred = predict(X_test)

print(f"{round(time()-start, 2)}sec")

/var/folders/s5/c05fkgj90cz2t3tmp495bdfc0000gn/T/ipykernel_63830/1591642778.py:4: DtypeWarning: Columns (60) have mixed types.Specify dtype option on import or set low_memory=False.
  df_1 = sort_csv_files(path)


109.14sec


In [69]:
print_score(Y_test, y_pred)

R²: 0.7389615421530696
RMSE: 0.5112532824525546
MSE: 0.2613799188185116


In [70]:
pkl_file_path = "saved_model.pkl"
save_model(pkl_file_path)
xgboost = load_model("saved_model.pkl")

In [71]:
print_score(Y_test, y_pred)

R²: 0.7389615421530696
RMSE: 0.5112532824525546
MSE: 0.2613799188185116


In [21]:
#feature engineering: nb de pieces à la place de plusieurs champs
#feature engineering sur la surface
#display(HTML(dept_75.describe().to_html()))

In [22]:
#df

In [44]:
#Cesar
from sklearn.preprocessing import MinMaxScaler
scaler = StandardScaler()
mutation['valeurfonc'] = scaler.fit_transform(mutation['valeurfonc'].values.reshape(-1, 1))

In [26]:
features = ["anneemut", "moismut", "nblot", "sterr", "nbvolmut", "nblocmut",
            "nblocmai", "nblocapt", "nblocdep", "nblocact", "latitude", "longitude",
            "arro", "surf"
           ]
X = mutation[features]
y = mutation["valeurfonc"]

In [28]:
mutation.head()

,Unnamed: 0,idmutation,idmutinvar,idopendata,idnatmut,datemut,libnatmut,valeurfonc,nbdispo,nblot,...,codservch,refdoc,anneemut,moismut,coddep,vefa,index,geometry,surf,arro
0,0,6539278.0,9b7b4d9b9700ca770f3f498b5d69eaf3,9b7b4d9b9700ca770f3f498b5d69eaf3,1.0,2018-04-06,Vente,2.105031,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.0,2
1,1,6660956.0,3c49a78206ac11f9695bad48967f898e,3c49a78206ac11f9695bad48967f898e,1.0,2017-07-06,Vente,0.625292,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,11
2,2,6374603.0,43db5a7809b243060479e1bca9def379,43db5a7809b243060479e1bca9def379,1.0,2019-07-18,Vente,-0.879954,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11
3,3,6393753.0,7bba166c612dfbd6c0694a2ab3209bab,7bba166c612dfbd6c0694a2ab3209bab,1.0,2019-05-16,Vente,0.816869,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,130.0,4
4,4,7090098.0,24b9ace5aaea518ba15bd43c0ffc6141,24b9ace5aaea518ba15bd43c0ffc6141,1.0,2014-05-14,Vente,-0.962058,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,8


In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 100)

In [30]:
# Cesar
#labelencoder = LabelEncoder()
#df['vefa'] =labelencoder.fit_transform(df['vefa'])
#df['l_section'] =labelencoder.fit_transform(df['l_section'])
#df['l_idpar'] =labelencoder.fit_transform(df['l_idpar'])

In [31]:
#Cesar
#Y = df['valeurfonc']
#X = df.drop('valeurfonc', axis=1)
# Split features and target into train and test sets


## Model

In [32]:
import sklearn
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import BaggingRegressor

### Scoring

In [33]:
# Let's define a function for each metrics
# R²
def rsqr_score(test, pred):
    """Calculate Root Mean Square Error score 

    Args:
        test -- test data
        pred -- predicted data

    Returns:
        Root Mean Square Error score
    """
    rsqr_ = r2_score(test, pred)
    return rsqr_
def rmse_score(test, pred):
    """Calculate Root Mean Square Error score 

    Args:
        test -- test data
        pred -- predicted data

    Returns:
        Root Mean Square Error score
    """
    rmse_ = np.sqrt(mean_squared_error(test, pred))
    return rmse_

def mse_score(test, pred):
    """Calculate Root Mean Square Error score 

    Args:
        test -- test data
        pred -- predicted data

    Returns:
        Root Mean Square Error score
    """
    mse_ = mean_squared_error(test, pred)
    return mse_

# Print the scores
def print_score(test, pred):
    """Print calculated score 

    Args:
        test -- test data
        pred -- predicted data

    Returns:
        print the regressor name
        print the R squared score
        print Root Mean Square Error score
        print Mean Square Error score
    """

    #print(f"- Regressor: {regr.__class__.__name__}")
    print(f"R²: {rsqr_score(test, pred)}")
    print(f"RMSE: {rmse_score(test, pred)}\n")
    print(f"MSE: {mse_score(test, pred)}\n")


### Model predicitions

In [37]:
X_train#.dropna()

,anneemut,moismut,nblot,sterr,nbvolmut,nblocmut,nblocmai,nblocapt,nblocdep,nblocact,latitude,longitude,arro,surf
1311819,2017.0,12.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,48.884278,2.298598,17,18.0
1611442,2017.0,7.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,48.564242,2.623987,0,178.0
38041,NaN,NaN,4.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,48.865350,2.370559,11,160.0
478617,NaN,NaN,0.0,975.0,0.0,1.0,1.0,0.0,0.0,0.0,48.945488,1.531158,0,212.0
157506,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,48.888805,2.360280,18,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230323,NaN,NaN,2.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,48.884426,2.581790,0,172.0
71054,NaN,NaN,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,48.863723,2.397150,20,112.0
228097,NaN,NaN,0.0,135765.0,0.0,0.0,0.0,0.0,0.0,0.0,48.846673,2.910354,0,0.0
1155840,NaN,NaN,0.0,240.0,0.0,1.0,1.0,0.0,0.0,0.0,49.043187,2.090337,0,110.0


In [34]:
bregr = BaggingRegressor()
bregr.fit(X_train, Y_train)
y_pred = bregr.predict(X_test)
print_score(Y_test, y_pred)
rmse_score(Y_test, y_pred)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [35]:
lgbm = LGBMRegressor()

lgbm.fit(X_train, Y_train)
y_pred = lgbm.predict(X_test)
print_score(Y_test, y_pred)


R²: 0.7192679762867746
RMSE: 0.5301879342074461

MSE: 0.2810992455791593



In [38]:
xgboost = XGBRegressor()
xgboost.fit(X_train, Y_train)
y_pred = xgboost.predict(X_test)
print_score(Y_test, y_pred)

R²: 0.7389615421530696
RMSE: 0.5112532824525546

MSE: 0.2613799188185116



In [40]:
rdf = RandomForestRegressor()

rdf.fit(X_train, Y_train)
y_pred = rdf.predict(X_test)
print_score(Y_test, y_pred)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [41]:
from sklearn.model_selection import GridSearchCV

# Define hyperparameters

param_grid = {
    'n_estimators': [10, 20, 50],
    'max_features': [0.5, 0.7, 1.0],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False]
}
model = LGBMRegressor()
# GridSearch
ridge_cv = GridSearchCV(model, param_grid = param_grid, cv=10, n_jobs=-1, verbose=1)

# fit the GridSearch on train set
ridge_cv.fit(X_train, Y_train)

# print best params and the corresponding R²
print(f"Best hyperparameters: {ridge_cv.best_params_}")
print(f"Best R² (train): {ridge_cv.best_score_}")


Fitting 10 folds for each of 36 candidates, totalling 360 fits
[LightGBM] [Warning] Unknown parameter: bootstrap_features
[LightGBM] [Warning] Unknown parameter: bootstrap
[LightGBM] [Warning] Unknown parameter: max_features
Best hyperparameters: {'bootstrap': True, 'bootstrap_features': True, 'max_features': 0.5, 'n_estimators': 50}
Best R² (train): 0.6949470171645635


In [43]:
y_pred = ridge_cv.predict(X_test)
print_score(Y_test, y_pred)

R²: 0.6959161446332858
RMSE: 0.5517985352067695

MSE: 0.3044816234563364

